In [135]:
from docplex.mp.model import Model
import numpy as np
from source_torch.mlca.mlca_nn import MLCA_NN
from source_torch.mlca.mlca_nn_mip import MLCA_NNMIP
from collections import OrderedDict

In [134]:
#random bundles of 1 or 0 
bundles = np.random.randint(2, size=(5, 10))
values = np.random.randint(100, size=(5, 1))
#print(bundles)
#print(values)
model = MLCA_NN(X_train = bundles, Y_train=values)

model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'device': 'cpu', 'regularization_type': 'l2', 'regularization': 0.01})

model.fit(epochs=10, batch_size=32)

models = OrderedDict([(0, model)])

mip = MLCA_NNMIP(models, L = 6000)

mip.solve_mip()

TypeError: 'BidderNN' object does not support indexing

In [4]:
def _get_model_weights(model): # torch
    model = model.model
    weights = []
    for params in model.parameters():
        weights.append(params.detach().cpu().numpy().T)        
    return weights

def _get_model_layer_shapes(model, layer_type=None):
    ''' return layer output shapes instead, 
        if 'input' is given as desired layer type, insert input dim at the beginning.
        assumes torch model '''
    # nnmodel = self.Models[key]
    nnmodel = model.model
    Layer_shapes = []
    for i, (name, param) in enumerate(nnmodel.named_parameters()):
        if (i==0) and ('input' in layer_type): 
            Layer_shapes.append(param.shape[1])
        if any([x in name for x in layer_type]) and ('bias' in name):
            Layer_shapes.append(param.shape[0])
    return Layer_shapes


def _clean_weights(Wb):
    for v in range(0, len(Wb)-2, 2):
        Wb[v][abs(Wb[v]) <= 1e-8] = 0
        Wb[v+1][abs(Wb[v+1]) <= 1e-8] = 0
        zero_rows = np.where(np.logical_and((Wb[v] == 0).all(axis=0), Wb[v+1] == 0))[0]
        if len(zero_rows) > 0:
            logging.debug('Clean Weights (rows) %s', zero_rows)
            Wb[v] = np.delete(Wb[v], zero_rows, axis=1)
            Wb[v+1] = np.delete(Wb[v+1], zero_rows)
            Wb[v+2] = np.delete(Wb[v+2], zero_rows, axis=0)
    return (Wb)

In [ ]:
MLCA_NNMIP

In [67]:
def galo_nn_ilp(nn_model):
    layer = 1
    Wb = _clean_weights(_get_model_weights(nn_model))
    key = nn_model#do we need this?
    m = Model(name='galo_nn')
    z = {}
    q = {}
    y = {}
    constraints = []
    for v in range(0, len(Wb), 2):
        W = Wb[v].transpose()
        b = Wb[v + 1]
        R, J = W.shape
        if v == 0:
            z.update({(i, 0, j): m.binary_var(name="x({})_{}".format(i, j)) for j in range(0, J)})
        
        z.update({(i, layer, r): m.continuous_var(lb=0, name="z({},{})_{}".format(i, layer, r)) for r in range(0, R)})
        #s in paper
        q.update({(i, layer, r): m.continuous_var(lb=0, name="s({},{})_{}".format(i, layer, r)) for r in range(0, R)}) # it is different from the initial code
        y.update({(i, layer, r): m.binary_var(name="y({},{})_{}".format(i, layer, r)) for r in range(0, R)}) # it is different from the initial code

        
        for r in range(0, R):#a lot of elif conditions here do it normally for now
            print(m.sum(W[r, j]*z[(i, layer-1, j)] for j in range(0, J)) + b[r] == z[(i, layer, r)] - q[(i, layer, r)])
            m.add_constraint(ct=(m.sum(W[r, j]*z[(i, layer-1, j)] for j in range(0, J)) + b[r] == z[(i, layer, r)] - q[(i, layer, r)]),
                                            ctname="AffineCT_Bidder{}_Layer{}_Row{}".format(i, layer, r))
            m.add_constraint(ct=(q[(i, layer, r)] <= nn_model.upper_bounds_s[key][layer][r][0]*y[(i, layer, r)]),
            
            m.add_constraint(ct=self.z[(i, layer, r)] <= self.y[(i, layer, r)]*self.upper_bounds_z[key][layer][r][0], ctname="BinaryCT_Bidder{}_Layer{}_Row{}_Z".format(i, layer, r))
            
            m.add_constraint(ct=self.s[(i, layer, r)] <= (1-self.y[(i, layer, r)])*self.upper_bounds_s[key][layer][r][0], ctname="BinaryCT_Bidder{}_Layer{}_Row{}_S".format(i, layer, r))
        layer = layer + 1

model = MLCA_NN(X_train = bundles, Y_train=values)
model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'device': 'cpu', 'regularization_type': 'l2', 'regularization': 0.01})
model.fit(epochs=10, batch_size=32)
galo_nn_ilp(model)


-0.590x(1)_0+0.055x(1)_1-0.383x(1)_2+0.530x(1)_3-0.091x(1)_4 == z(1,1)_0-s(1,1)_0
0.164x(1)_0-0.595x(1)_1-0.334x(1)_2+0.236x(1)_3-0.404x(1)_4 == z(1,1)_1-s(1,1)_1
-0.179x(1)_0-0.192x(1)_1-0.507x(1)_2+0.100x(1)_3-0.207x(1)_4 == z(1,1)_2-s(1,1)_2
0.240x(1)_0-0.478x(1)_1-0.055x(1)_2+0.177x(1)_3-0.356x(1)_4 == z(1,1)_3-s(1,1)_3
-0.272x(1)_0+0.473x(1)_1+0.521x(1)_2-0.564x(1)_3+0.484x(1)_4 == z(1,1)_4-s(1,1)_4
-0.111x(1)_0-0.348x(1)_1-0.421x(1)_2+0.382x(1)_3+0.269x(1)_4 == z(1,1)_5-s(1,1)_5
-0.567x(1)_0-0.368x(1)_1+0.497x(1)_2+0.210x(1)_3-0.220x(1)_4 == z(1,1)_6-s(1,1)_6
0.535x(1)_0-0.062x(1)_1+0.250x(1)_2-0.482x(1)_3+0.068x(1)_4 == z(1,1)_7-s(1,1)_7
0.452x(1)_0+0.337x(1)_1+0.002x(1)_2+0.354x(1)_3+0.440x(1)_4 == z(1,1)_8-s(1,1)_8
0.208x(1)_0-0.021x(1)_1+0.420x(1)_2-0.215x(1)_3-0.571x(1)_4 == z(1,1)_9-s(1,1)_9
0.221z(1,1)_0+0.285z(1,1)_1-0.153z(1,1)_2+0.241z(1,1)_3-0.473z(1,1)_4+0.266z(1,1)_5+0.301z(1,1)_6+0.279z(1,1)_7+0.339z(1,1)_8+0.491z(1,1)_9 == z(1,2)_0-s(1,2)_0
0.047z(1,1)_0+0.047z(1,1

In [57]:
count = 0
M = 8
#big integer
C = 10000
L = 10000

while count < 10:
    model = MLCA_NN(X_train = bundles, Y_train=values)
    model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'device': 'cpu', 'regularization_type': 'l2', 'regularization': 0.01})
    model.fit(epochs=10, batch_size=32)
    weights = _get_model_weights(model)
    layers = _get_model_layer_shapes(model, layer_type=['dense', 'input'])
    Wb = _clean_weights(weights)

    for i in range(len(values)):
        
        m  = Model(name='galo_nn')

        #create z^k variables
        z = {}
        y = {}
        q = {}


        
        for i, layer in enumerate(layers):
            y.update{(i, layer, r): self.Mip.binary_var(name="y({},{})_{}".format(i, layer, r))]
            y.append(m.binary_var_list(range(layer), name=f'y_{i}'))
            k.append(m.continuous_var_list(range(layer), name=f'k_{i}'))
            q.append(m.continuous_var_list(range(layer), name=f'q_{i}'))
            if i == 0:
                z.append(m.binary_var_list(range(layer), name=f'z_{i}'))
            else:
                z.append(m.continuous_var_list(range(layer), name=f'z_{i}'))

        #print(z)
        r = m.continuous_var(name='r')
        #x = m.binary_var_list(range(M), name='x')
        b = m.binary_var_list(range(len(values)), name='b')
        
        constraints = []
        constraints.append(values[i] - z[-1][0] <= r)
        constraints.append(z[-1][0] - values[i] <= r)
        
        for i in range(len(values)):
            constraints.append(values[i] - z[-1][0] + C * b[i] <= r)
            constraints.append(z[-1][0] - values[i] + C * (1- b[i]) <= r)
        
        #constraints.append(z[0] == x)
        # for k in range(M):
        #     m.add_constraint(x[k] == z[0][k])
        #     #constraints.append(x[k] = z[0][k])


        #W = Wb[v].transpose()
        #b = Wb[v + 1]
        #R, J = W.shape
        #print('R: ' + str(R))
        #print('J: ' + str(J))

        #print(b)



        for layer in range(0, len(Wb), 2):#should it start with 1?
            W = Wb[layer].transpose()
            bias = Wb[layer + 1]
            #print(W)
            R, J = W.shape
            print('R: ' + str(R))
            print('J: ' + str(J))
            print(z)
            for row in range(0, R):
                print('bias' + str(bias[row]))
                print(z[layer][row])
                print(q[layer][row])
                m.add_constraint(z[layer][row] - q[layer][row] == m.sum(W[layer-1][j] *z[layer-1][j]) + bias[row])
                print('Layer: ' + str(layer))
                print('Row: ' + str(row))

                #ak = z[layer][row] - q[layer][row] == (m.sum(W[layer-1][j] *z[layer-1][j]) for j in range(0, J)) + bias[row]
                #print(ak)
                #constraints.append(z[k][r] - q[k][r] == m.sum(W[k-1][r] *z[k-1][r]) + b[k-1][j])
            # for j in range(layers[k]):
            #     print('Weights')
            #     #print(Wb[k-1][j])
            #     #print('z')
            #     #print(z[k-1][j])
            #     constraints.append(z[k][j] - q[k][j] == c_weights[k-1][j] *z[k-1][j] + b[k-1][j])
            #     constraints.append(0 <= z[k][j] <= y[k][j] * L)
            #     constraints.append(0 <= q[k][j] <= (1-y[k][j]) * L)
    count += 1
        

    

R: 10
J: 5
[[docplex.mp.Var(type=B,name='z_0_0'), docplex.mp.Var(type=B,name='z_0_1'), docplex.mp.Var(type=B,name='z_0_2'), docplex.mp.Var(type=B,name='z_0_3'), docplex.mp.Var(type=B,name='z_0_4')], [docplex.mp.Var(type=C,name='z_1_0'), docplex.mp.Var(type=C,name='z_1_1'), docplex.mp.Var(type=C,name='z_1_2'), docplex.mp.Var(type=C,name='z_1_3'), docplex.mp.Var(type=C,name='z_1_4'), docplex.mp.Var(type=C,name='z_1_5'), docplex.mp.Var(type=C,name='z_1_6'), docplex.mp.Var(type=C,name='z_1_7'), docplex.mp.Var(type=C,name='z_1_8'), docplex.mp.Var(type=C,name='z_1_9')], [docplex.mp.Var(type=C,name='z_2_0'), docplex.mp.Var(type=C,name='z_2_1'), docplex.mp.Var(type=C,name='z_2_2'), docplex.mp.Var(type=C,name='z_2_3'), docplex.mp.Var(type=C,name='z_2_4'), docplex.mp.Var(type=C,name='z_2_5'), docplex.mp.Var(type=C,name='z_2_6'), docplex.mp.Var(type=C,name='z_2_7'), docplex.mp.Var(type=C,name='z_2_8'), docplex.mp.Var(type=C,name='z_2_9')], [docplex.mp.Var(type=C,name='z_3_0'), docplex.mp.Var(type

IndexError: list index out of range